## Part 4: Customer Churn Prediction
Notes from this notebook comes from various resources listed at the end of the notebook. This will be good exposure to using more plotly to produce interactable plots vs. matplotlib as well predictive modeling.

-------

### Areas of Focus

### Results

### Issues

### What is churning?

### Process

### Design

### Libraries

### Helpers

### Data

In [ ]:
# TODO: Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components=2).fit(good_data)

# TODO: Transform the good data using the PCA fit above
reduced_data = pca.transform(good_data)

# TODO: Transform log_samples using the PCA fit above
pca_samples = pca.transform(log_samples)

# Create a DataFrame for the reduced data
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

# Display sample log-data after applying PCA transformation in two dimensions
display(pd.DataFrame(np.round(pca_samples, 4), columns = ['Dimension 1', 'Dimension 2']))

# Create a biplot
vs.biplot(good_data, reduced_data, pca)